In [87]:
import pandas as pd
import numpy as np
import calendar
from dateutil.relativedelta import relativedelta
from datetime import datetime

dataset = pd.read_excel(r'C:\Users\ssing296\Desktop\Natividad\FCOTTest.xlsx')
    
dataset.sort_values(by=['ActualOperatingRoom','PatientInRoomActualDatetime'], ascending=True, inplace=True)

def fcotlogic(dataset):

    
    firstcasestrt = '07:00:00'
    firstcasestp =  '09:30:00'
    FMT = '%H:%M:%S'

    intime_list = dataset["PatientInRoomActualDatetime"].tolist()
    
    casedate_list = (dataset["CaseDateonly"]).tolist()
    
    #print(casedate_list)
    act_or_list = dataset["ActualOperatingRoom"].tolist()

    casetype_list = dataset['CaseType'].tolist() 

    #print(len(intime_list))
    #print("------------------------------------")
    
    fc_start = []
    fc_end = []
    
    for i in range(0,len(intime_list)):
        if(pd.isnull(intime_list[i])):
            A = intime_list[i-1].date().strftime("%d/%m/%y")+" "+firstcasestrt
            B = intime_list[i-1].date().strftime("%d/%m/%y")+" "+firstcasestp
            fc_start.append(datetime.strptime(A, '%d/%m/%y %H:%M:%S'))
            fc_end.append(datetime.strptime(B, '%d/%m/%y %H:%M:%S'))
        else:
            A = intime_list[i].date().strftime("%d/%m/%y")+" "+firstcasestrt
            B = intime_list[i].date().strftime("%d/%m/%y")+" "+firstcasestp
            fc_start.append(datetime.strptime(A, '%d/%m/%y %H:%M:%S'))
            fc_end.append(datetime.strptime(B, '%d/%m/%y %H:%M:%S'))

    fcot_flag = []

    if(casetype_list[0].lower() == "planned"  and (intime_list[0] >= fc_start[0]) and (intime_list[0] <= fc_end[0])):
        fcot_flag.append("FC") 
    else:
        fcot_flag.append("TFC") 
    
    #print(len(fc_start))
    #print(len(fc_end))
    for i in range(1,len(intime_list)):
        casetimecurr = intime_list[i]
        casetimepre = intime_list[i-1]
        casedatecurr = casedate_list[i]
        casedatepre = casedate_list[i-1]
        orcurr = act_or_list[i]
        orpre = act_or_list[i-1]
        casetype = casetype_list[i].lower()
        
#        if(casetype[i].lower() == "planned"  and (intime_list[i] >= fc_start[i]) and (intime_list[i] <= fc_end[i])):
#            fcot_flag.append("FCOT") 
#        else:
#            fcot_flag.append("TFC")
            

        if casetype != "planned":
            fcot_flag.append("TFC")
        elif casetype == "planned":
            if casetimecurr >= fc_start[i] and casetimecurr <= fc_end[i]:
                if casedatecurr != casedatepre:
                    fcot_flag.append("FC") 
                else:
                    if orcurr != orpre:
                        fcot_flag.append("FC") 
                    else:
                        if casetimepre <fc_start[i]:
                            fcot_flag.append("FC") 
                        else:
                            fcot_flag.append("TFC") 
            else:

                fcot_flag.append("TFC")             
            
            
    dataset["FCOT"] = fcot_flag
    
fcotlogic(dataset)
print(dataset[["SurgicalCaseID","CaseDate","ActualOperatingRoom","PatientInRoomActualDatetime","FCOT"]])

    SurgicalCaseID            CaseDate ActualOperatingRoom  \
0            25604 2021-05-14 08:30:00               OR.01   
1            25703 2021-05-14 16:30:00               OR.01   
2            25564 2021-05-14 07:30:00               OR.02   
3            25632 2021-05-14 08:45:00               OR.02   
4            25424 2021-05-14 11:30:00               OR.02   
5            25706 2021-05-14 20:00:00               OR.02   
6            25708 2021-05-14 23:00:00               OR.02   
7            25462 2021-05-14 09:30:00               OR.03   
8            25457 2021-05-14 07:30:00               OR.03   
9            25460 2021-05-14 08:30:00               OR.03   
10           25629 2021-05-14 10:30:00               OR.03   
11           25664 2021-05-14 14:00:00               OR.03   
12           25504 2021-05-14 10:00:00               OR.03   
13           25631 2021-05-14 15:00:00               OR.05   
14           25707 2021-05-14 06:00:00               OR.05   

   Pati